In [1]:
import stackstac
import satsearch
from datetime import datetime
import dask.diagnostics

import rioxarray
import xarray as xr
import numpy as np
import geopandas as gpd
import geojson
import json

from datetime import datetime

from shapely.geometry import mapping

import zarr

In [2]:
#!pip install dask distributed --upgrade

In [4]:
#import dask
from dask.distributed import Client
client = Client()

In [5]:
client

Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 127.77 GiB
Comm: tcp://127.0.0.1:64704,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 127.77 GiB
Comm: tcp://127.0.0.1:64727,Total threads: 3
Dashboard: http://127.0.0.1:64728/status,Memory: 31.94 GiB
Nanny: tcp://127.0.0.1:64706,


In [6]:
bbox = [150.53718391857018, -34.47320307771639, 151.25748275818813,  -33.82982205634835]                        

In [11]:
import os
%env AWS_NO_SIGN_REQUEST=Yes

t0 = datetime.now()
stac_found = satsearch.Search(
    url="https://explorer.sandbox.dea.ga.gov.au/stac/",
    bbox=bbox,
    collections=["s1_gamma0_geotif_scene"],
    datetime="2000-02-01/2021-03-31",
).found()  
print(stac_found)

stac_items = satsearch.Search(
    url="https://explorer.sandbox.dea.ga.gov.au/stac/",
    bbox=bbox,
    collections=["s1_gamma0_geotif_scene"],
    datetime="2000-02-01/2021-03-31",
).items()

print('searching time: {}'.format(datetime.now()-t0))

print("Num of Items", len(stac_items))

t0 = datetime.now()
stack = stackstac.stack(stac_items, epsg=6933, resolution=20480, resampling=1)

print('lazy stacking time: {}'.format(datetime.now()-t0))

print("Stack Size in GB: ", round(stack.nbytes/1e9,0))

stack


env: AWS_NO_SIGN_REQUEST=Yes
142
searching time: 0:00:01.418115
Num of Items 142
lazy stacking time: 0:00:00.013252
Stack Size in GB:  0.0


,Array,Chunk
Bytes,2.69 MiB,9.69 kiB
Shape,"(142, 2, 31, 40)","(1, 1, 31, 40)"
Count,853 Tasks,284 Chunks
Type,float64,numpy.ndarray


In [12]:
stack.where(stack['time']==1483298616446879000).plot()

RuntimeError: Error opening 's3://dea-public-data/odc-conf/sentinel_examples/Sentinel-1/C-SAR/GRD/2017/2017-01/30S145E-35S150E/S1A_IW_GRDH_1SDV_20170101T192323_20170101T192348_014642_017CF2_876C.data/Gamma0_VH.tif': RasterioIOError("'/vsis3/dea-public-data/odc-conf/sentinel_examples/Sentinel-1/C-SAR/GRD/2017/2017-01/30S145E-35S150E/S1A_IW_GRDH_1SDV_20170101T192323_20170101T192348_014642_017CF2_876C.data/Gamma0_VH.tif' not recognized as a supported file format.")

In [9]:
stack[0]

<xarray.DataArray 'stackstac-89c69c76729595e9aa1ce815a0e4ff84' (band: 2, y: 31, x: 40)>
dask.array<getitem, shape=(2, 31, 40), dtype=float64, chunksize=(1, 31, 40), chunktype=numpy.ndarray>
Coordinates: (12/13)
    time                  datetime64[ns] 2017-01-01T19:23:36.446879
    id                    <U36 'dd98a265-b605-453d-95a1-48e47506512b'
  * band                  (band) <U2 'vh' 'vv'
  * x                     (x) float64 1.413e+07 1.415e+07 ... 1.493e+07
  * y                     (y) float64 -3.809e+06 -3.83e+06 ... -4.424e+06
    title                 <U67 'S1A_IW_GRDH_1SDV_20170101T192323_20170101T192...
    ...                    ...
    odc:product           <U22 's1_gamma0_geotif_scene'
    instruments           <U3 'sar'
    proj:epsg             int32 4326
    platform              <U10 'sentinel-1'
    cubedash:region_code  object None
    epsg                  int32 6933
Attributes:
    spec:        RasterSpec(epsg=6933, bounds=(14131200, -4444160, 14950400, ...
    crs:         epsg:6933
    transform:   | 20480.00, 0.00, 14131200.00|\n| 0.00,-20480.00,-3809280.00...
    resolution:  20480

In [10]:
stack.sel(band='vv')[0].plot()

RuntimeError: Error opening 's3://dea-public-data/odc-conf/sentinel_examples/Sentinel-1/C-SAR/GRD/2017/2017-01/30S145E-35S150E/S1A_IW_GRDH_1SDV_20170101T192323_20170101T192348_014642_017CF2_876C.data/Gamma0_VV.tif': RasterioIOError("'/vsis3/dea-public-data/odc-conf/sentinel_examples/Sentinel-1/C-SAR/GRD/2017/2017-01/30S145E-35S150E/S1A_IW_GRDH_1SDV_20170101T192323_20170101T192348_014642_017CF2_876C.data/Gamma0_VV.tif' not recognized as a supported file format.")

In [13]:
stack.sel(band='vv').plot()

RuntimeError: Error opening 's3://dea-public-data/odc-conf/sentinel_examples/Sentinel-1/C-SAR/GRD/2017/2017-04/30S150E-35S155E/S1A_IW_GRDH_1SDV_20170426T191505_20170426T191530_016319_01B008_0D8E.data/Gamma0_VV.tif': RasterioIOError("'/vsis3/dea-public-data/odc-conf/sentinel_examples/Sentinel-1/C-SAR/GRD/2017/2017-04/30S150E-35S155E/S1A_IW_GRDH_1SDV_20170426T191505_20170426T191530_016319_01B008_0D8E.data/Gamma0_VV.tif' not recognized as a supported file format.")

In [ ]:
type(stac_items)
stac_items[0]